# TODO
* Xception
* DenseNet
* NasNetLarge

# TODO

# Implementation

## Get images & captions files

In [ ]:
!cp -r drive/MyDrive/LICENTA-DATASET/flickr30k-images.tar.gz .
!tar -xf flickr30k-images.tar.gz
%rm flickr30k-images/readme.txt
!wget https://raw.githubusercontent.com/dima331453/Flickr30k-Romanian/main/flickr30k_romanian.tsv
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.vec.gz

--2024-06-17 13:10:31--  https://raw.githubusercontent.com/dima331453/Flickr30k-Romanian/main/flickr30k_romanian.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25618304 (24M) [text/plain]
Saving to: ‘flickr30k_romanian.tsv’

flickr30k_romanian. 100%[===================>]  24.43M  --.-KB/s    in 0.05s   

2024-06-17 13:10:34 (474 MB/s) - ‘flickr30k_romanian.tsv’ saved [25618304/25618304]

--2024-06-17 13:10:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.121, 108.157.254.15, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123365973

In [ ]:
!gzip -d cc.ro.300.vec.gz

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.bin.gz

--2024-06-17 13:12:10--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.121, 108.157.254.124, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4508185279 (4.2G) [application/octet-stream]
Saving to: ‘cc.ro.300.bin.gz’

cc.ro.300.bin.gz    100%[===================>]   4.20G  24.0MB/s    in 3m 5s   

2024-06-17 13:15:15 (23.3 MB/s) - ‘cc.ro.300.bin.gz’ saved [4508185279/4508185279]



## Imports

In [ ]:
!pip install keras-self-attention
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score
!pip install git+https://github.com/michelecafagna26/cider.git#egg=cidereval
!pip install vit-keras
!pip install tensorflow-addons

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18894 sha256=da1962d1a29a7a2513466860f25d237fc06db6eff8e93f7652e2a89dfe8c8463
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import os
from os import listdir, mkdir
import string
from random import shuffle
from collections import defaultdict
import random
from tqdm import tqdm
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.applications import ResNet50, ResNet50V2, InceptionV3, InceptionResNetV2, EfficientNetB4, EfficientNetB7

from keras.models import Sequential
from keras.layers import RandomFlip, RandomRotation, RandomContrast
from keras.layers import Reshape, Bidirectional, Flatten
from keras.models import Model
from keras.layers import Input, Embedding, Dropout, Dense
from keras.layers import LSTM, GRU
from keras.layers import Attention, AdditiveAttention
from keras_self_attention import SeqSelfAttention
from keras.layers import add, concatenate

from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import Adam

from keras.utils import plot_model

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import evaluate
from cidereval import cider, ciderD

from vit_keras import vit, utils

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
ter = evaluate.load("ter")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

cc_vec = load_vectors('./cc.ro.300.vec')

## Constants

In [ ]:
IMAGES_PATH = "flickr30k-images"
CAPTIONS_PATH = "./flickr30k_romanian.tsv"

# EMBEDS_PATH = "drive/MyDrive/LICENTA-DATASET/robert-embs.npy"
# TOKS_PATH = "drive/MyDrive/LICENTA-DATASET/robert-toks.npy"

MODELS = ["ResNet50", "ResNet50V2", "InceptionV3", "InceptionResNetV2", "EfficientNetB4", "EfficientNetB7", "VitB16"]
MODEL_NAME = "ResNet50" # @param ["ResNet50", "ResNet50V2", "InceptionV3", "InceptionResNetV2", "EfficientNetB4", "EfficientNetB7", "VitB16"]
MODELS_BASE = "./Models/"
FEATURES_BASE = "/features/"
TRAIN_BASE = "/train"
CONFIG_BASE = "/1/"
UTILS_BASE = "utils/"

FEATURES_PATH = MODELS_BASE + MODEL_NAME + FEATURES_BASE
CP_PATH = MODELS_BASE + MODEL_NAME + TRAIN_BASE + CONFIG_BASE

TOKENIZER_PATH = MODELS_BASE + UTILS_BASE + 'tokenizer.pkl'
DESCRIPTIONS_PATH = MODELS_BASE + UTILS_BASE + 'descriptions.txt'
TRAIN_PATH = MODELS_BASE + UTILS_BASE + 'train.txt'
VALID_PATH = MODELS_BASE + UTILS_BASE + 'valid.txt'
TEST_PATH = MODELS_BASE + UTILS_BASE + 'test.txt'

In [ ]:
IMAGE_SIZE = (380, 380)
IMAGE_AUGMENTATION = Sequential(
    [
        RandomFlip("horizontal"),
        RandomRotation(0.2),
        RandomContrast(0.3),
    ]
)

LR = 3e-4

EMBED_DIM = 128 # @param ["128", "256", "512"] {type:"raw"}
FF_DIM = 256 # @param ["128", "256", "512"] {type:"raw"}
LSTM_DIM = 256 # @param ["128", "256", "512"] {type:"raw"}
GRU_DIM = 256 # @param ["128", "256", "512"] {type:"raw"}
BATCH_SIZE = 32 # @param ["32", "64", "128"] {type:"raw"}
EPOCHS = 10
DROPOUT = 0.3


# VOCAB_SIZE = 10000
SEQ_LENGTH = 25
WORD_THRESHOLD = 2

In [ ]:
mkdir(MODELS_BASE)
mkdir(MODELS_BASE + UTILS_BASE)
for model in MODELS:
  mkdir(MODELS_BASE + model)
  mkdir(MODELS_BASE + model + FEATURES_BASE)
  mkdir(MODELS_BASE + model + TRAIN_BASE)
  mkdir(MODELS_BASE + model + TRAIN_BASE + CONFIG_BASE)

## Captions preprocessing

In [ ]:
# ( - start
# ) - end
# _ - unk
# # - pad


In [ ]:
def load_descriptions(filename):
    file = open(filename, 'r')
    doc = file.read()
    file.close()

    mapping = dict()
    first = True
    for line in doc.split('\n'):
        if first:
          first = False
          continue
        tokens = line.split('\t')
        if len(line) < 4:
            continue


        image_id, image_desc = tokens[1], tokens[3:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)

        if image_id not in mapping:
            mapping[image_id] = list()

        mapping[image_id].append(image_desc)
    return mapping

def clean_descriptions(descriptions, seq_len):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]

            desc = [word if word in cc_vec else '_' for word in desc]

            # store as string
            desc_list[i] = '( ' + ' '.join(desc[:seq_len]) + ' )'


def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

## Model variants

### GRU + CONCATENATE

In [ ]:
def define_model_GRU_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, gru_dim, dropout):
    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = GRU(gru_dim)(se2)

    decoder1 = concatenate([fe2, se3])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### GRU + ADD

In [ ]:
def define_model_GRU_ADD(vocab_size, max_length, in_shape, embed_dim, ff_dim, gru_dim, dropout):
    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = GRU(gru_dim)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### LSTM + CONCATENATE

In [ ]:
def define_model_LSTM_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, lstm_dim, dropout):
    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = LSTM(lstm_dim)(se2)

    decoder1 = concatenate([fe2, se3])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### LSTM + ADD

In [ ]:
def define_model_LSTM_ADD(vocab_size, max_length, in_shape, embed_dim, ff_dim, lstm_dim, dropout):
    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = LSTM(lstm_dim)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### GRU + ATT Luong + CONCATENATE

In [ ]:
def define_model_GRU_ATTLuong_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, gru_dim, dropout):

    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = Bidirectional(GRU(gru_dim, return_sequences=True))(se2)
    se4 = SeqSelfAttention(attention_type="multiplicative")(se3)
    se5 = Flatten()(se4)
    se6 = Dense(ff_dim, activation='relu')(se5)

    decoder1 = concatenate([fe2, se6])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### LSTM + ATT Luong + CONCATENATE

In [ ]:
def define_model_LSTM_ATTLuong_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, lstm_dim, dropout):

    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = Bidirectional(LSTM(lstm_dim, return_sequences=True))(se2)
    se4 = SeqSelfAttention(attention_type="multiplicative")(se3)
    se5 = Flatten()(se4)
    se6 = Dense(ff_dim, activation='relu')(se5)

    decoder1 = concatenate([fe2, se6])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### GRU + ATT Bahdanau + CONCATENATE

In [ ]:
def define_model_GRU_ATTBahdanau_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, gru_dim, dropout):

    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = Bidirectional(GRU(gru_dim, return_sequences=True))(se2)
    se4 = SeqSelfAttention(attention_type="additive")(se3)
    se5 = Flatten()(se4)
    se6 = Dense(ff_dim, activation='relu')(se5)

    decoder1 = concatenate([fe2, se6])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

### LSTM + ATT Bahdanau + CONCATENATE

In [ ]:
def define_model_LSTM_ATTBahdanau_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dim, lstm_dim, dropout):

    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embed_dim, mask_zero=True)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = Bidirectional(LSTM(lstm_dim, return_sequences=True))(se2)
    se4 = SeqSelfAttention(attention_type="additive")(se3)
    se5 = Flatten()(se4)
    se6 = Dense(ff_dim, activation='relu')(se5)

    decoder1 = concatenate([fe2, se6])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    return model

## Model functions

In [ ]:
def define_model(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout, variant = "v1"):
    if variant == "v1":
      model = define_model_LSTM_ADD(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v2":
      model = define_model_LSTM_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v3":
      model = define_model_GRU_ADD(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v4":
      model = define_model_GRU_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v5":
      model = define_model_GRU_ATTLuong_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v6":
      model = define_model_LSTM_ATTLuong_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v7":
      model = define_model_GRU_ATTBahdanau_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    elif variant == "v8":
      model = define_model_LSTM_ATTBahdanau_CONC(vocab_size, max_length, in_shape, embed_dim, ff_dimm, lstm_or_gru_dim, dropout)
    else:
      model = None
    # model.summary()
    plot_model(model, to_file=f'model_{variant}.png', show_shapes=True)

    return model

def get_cnn_model(model_name):
    model = None

    if model_name == "ResNet50":
      model = ResNet50()
    elif model_name == "ResNet50V2":
      model = ResNet50V2()
    elif model_name == "InceptionV3":
      model = InceptionV3()
    elif model_name == "InceptionResNetV2":
      model = InceptionResNetV2()
    elif model_name == "EfficientNetB4":
      model = EfficientNetB4()
    elif model_name == "EfficientNetB7":
      model = EfficientNetB7()
    else:
      model = None

    model.trainable = False
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output) #  put -2
    # model.summary()

    return model, model.layers[-1].output_shape[-1]

def get_vit_model():

    model = vit.vit_b16(
        activation='sigmoid',
        pretrained=True,
        include_top=True,
        pretrained_top=True
    )

    model.trainable = False
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output) #  put -2
    # model.summary()

    return model, model.layers[-1].output_shape[-1]

## Dataset generator

In [ ]:
def create_sequences(max_length, desc_list, photo, vocab_size, vocab):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    seqs = [desc.split() for desc in desc_list]
    for seq in seqs:
      for i in range(1, len(seq)):
          # split into input and output pair
          in_seq, out_seq = seq[:i], seq[i]
          # pad input sequence
          in_seq = ["#"] * (max_length - i) + in_seq
          in_seq = [vocab[w] for w in in_seq]
          # encode output sequence
          out_seq = vocab[out_seq]

          out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
          # store
          X1.append(photo)
          X2.append(in_seq)
          y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)

def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # img = img_aug(img)
    return img

def augment_and_extract_features(img, img_aug):
    img = img_aug(img)
    return cnn_model(img)

def process_input(img_path, key):
    return preprocess_image(img_path), key

def data_generator(descriptions, cnn_model, max_length, batch_size, directory, img_aug, vocab_size, vocab):
    image_paths = [os.path.join(directory, key + '.jpg') for key in descriptions.keys()]

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, list(descriptions.keys())))
    dataset = dataset.map(process_input, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    dataset = dataset.repeat()


    for batch_images, batch_keys in dataset:
        img_features = augment_and_extract_features(batch_images, img_aug)
        batch_keys = [key.decode('utf-8') for key in batch_keys.numpy()]  # Convert bytes to string
        photos = {key: feature for key, feature in zip(batch_keys, img_features.numpy())}
        in_img_batch, in_seq_batch, out_word_batch = [], [], []

        for key in batch_keys:
            desc_list = descriptions[key]
            photo = photos[key]  # Adjust to remove the extra dimension
            in_img, in_seq, out_word = create_sequences(max_length, desc_list, photo, vocab_size, vocab)

            in_img_batch.append(in_img)
            in_seq_batch.append(in_seq)
            out_word_batch.append(out_word)

        yield [[np.vstack(in_img_batch), np.vstack(in_seq_batch)], np.vstack(out_word_batch)]

# Prepare for training

In [ ]:
descriptions = load_descriptions(CAPTIONS_PATH)
clean_descriptions(descriptions, SEQ_LENGTH)

all_items = list(descriptions.keys())
np.random.seed(42)
random.seed(42)
random.shuffle(all_items)

train_items = set(all_items[:-5000])
train_descriptions =  {key: value for key, value in descriptions.items() if key in train_items}

test_items = set(all_items[-5000:])
test_descriptions =  {key: value for key, value in descriptions.items() if key in test_items}

In [ ]:
# ( - start
# ) - end
# _ - unk
# # - pad

In [ ]:
vocab = {}
token_id = 0
if "#" not in vocab:
  vocab["#"] = token_id
  token_id += 1
# Iterate over the dictionary
for key, phrases in train_descriptions.items():
    for phrase in phrases:
        # Split the phrase into words
        words = phrase.split()
        for word in words:
            # Add the word to the vocab dictionary with a unique token ID if it's not already present
            if word not in vocab:
                vocab[word] = token_id
                token_id += 1
# Print the vocabulary dictionary
print(vocab)
vocab_size = len(vocab)
embedding_matrix = np.zeros((vocab_size, 300))
embedding_matrix[0] = [0.1396, 0.0781, 0.24, -0.1345, -0.0974, -0.0251, 0.2608, 0.2303, -0.1494, 0.1684, 0.1145, 0.285, 0.1465, -0.0674, 0.1872, 0.1032, -0.1943, -0.0851, -0.0285, -0.0074, 0.1352, 0.272, 0.1313, 0.0454, 0.2347, 0.0749, 0.1729, 0.1222, 0.0593, -0.0451, 0.0773, -0.1004, -0.0264, -0.1723, -0.1323, 0.0129, -0.0754, -0.5246, -0.1852, -0.0235, 0.0783, 0.1332, -0.0828, -0.1453, -0.0559, 0.0334, 0.1797, -0.0274, 0.148, -0.0029, 0.3841, 0.0892, 0.0125, -0.0239, -0.1819, 0.3117, -0.2164, 0.049, 0.0265, 0.0674, -0.0607, 0.0579, 0.0532, -0.1666, -0.135, -0.0068, -0.0121, -0.1632, -0.0833, 0.172, 0.1504, 0.0952, 0.0653, 0.2227, 0.0499, 0.0026, 0.0555, -0.1807, 0.2257, -0.1208, 0.04, -0.043, -0.2201, -0.1097, -0.0562, -0.2809, -0.0126, 0.1685, 0.3889, 0.0447, 0.0565, -0.1056, 0.2351, -0.2042, 0.2157, 0.1195, 0.153, -0.137, -0.1908, -0.0909, 0.1134, 0.0375, -0.008, 0.0072, -0.0671, -0.1341, 0.0578, -0.3204, 0.2108, 0.0361, 0.0574, -0.2162, -0.0742, -0.1123, -0.0456, 0.0833, -0.0114, -0.0344, -0.0322, -0.1444, 0.0868, 0.079, -0.0789, -0.0699, -0.0838, 0.074, 0.0553, -0.1033, -0.1181, -0.0432, 0.0737, -0.2369, 0.303, 0.1135, 0.1256, -0.154, -0.3039, 0.0021, 0.0169, -0.0731, -0.0186, 0.0562, -0.155, 0.252, 0.0197, -0.1254, 0.2048, -0.1188, -0.153, 0.1008, -0.0324, 0.3603, 0.0656, 0.0917, -0.1274, 0.01, 0.2438, -0.1789, -0.1586, 0.3359, 0.2284, 0.0288, -0.0898, 0.1444, 0.1017, -0.1702, -0.0968, 0.0271, 0.1789, 0.1627, -0.1669, 0.0503, -0.096, -0.024, 0.0206, 0.1191, -0.095, 0.1699, 0.0881, -0.0791, -0.0449, -0.0545, -0.1682, 0.0462, -0.1897, 0.0966, -0.1657, 0.0415, -0.3313, -0.1053, 0.0042, 0.1175, -0.1415, -0.1339, 0.0584, 0.0945, -0.1346, 0.0538, 0.0668, 0.0423, 0.0469, 0.3503, -0.0079, -0.1376, -0.0008, 0.0555, 0.109, -0.2525, 0.0096, 0.2079, 0.1312, 0.0047, -0.1484, -0.0147, 0.0074, 0.0667, -0.0618, -0.2579, 0.1085, 0.2515, 0.1119, -0.0746, 0.0994, 0.053, -0.0512, 0.0033, 0.1618, -0.0152, 0.1187, 0.0835, 0.0337, 0.0995, -0.0042, -0.1456, -0.1318, -0.1186, -0.1295, 0.2026, -0.0608, -0.0363, 0.2472, 0.026, 0.3144, 0.0881, -0.1134, 0.1414, -0.0384, 0.3088, -0.0308, 0.1326, 0.242, 0.0613, 0.2009, 0.1288, -0.188, -0.072, -0.353, 0.0737, -0.036, 0.0081, 0.1798, -0.0308, -0.3854, -0.2839, 0.0494, 0.1479, -0.1693, 0.0479, -0.0972, -0.1239, -0.1005, -0.0773, -0.2039, -1.1538, -0.3212, 0.0193, 0.3024, 0.1375, 0.3341, -0.1579, -0.1451, 0.0277, 0.1111, -0.1546, 0.1953, 0.1095, -0.0425, -0.3892, 0.1135, -0.2172, -0.2092, 0.245, 0.308, -0.0282, -0.011, -0.0915, 0.0876, 0.1371, -0.1585, 0.0002]
for word, i in vocab.items():
  if i == 0:
    continue
  embedding_matrix[i] = list(cc_vec[word])

{'#': 0, '(': 1, 'doi': 2, 'tineri': 3, 'cu': 4, 'păr': 5, 'șaten': 6, 'se': 7, 'uită': 8, 'la': 9, 'mâinile': 10, 'lor': 11, 'în': 12, 'timp': 13, 'ce': 14, 'stau': 15, 'curte': 16, ')': 17, 'bărbați': 18, 'albi': 19, 'sunt': 20, 'afară': 21, 'lângă': 22, 'multe': 23, 'tufe': 24, 'tricouri': 25, 'verzi': 26, 'întro': 27, 'un': 28, 'bărbat': 29, 'cămașă': 30, 'albastră': 31, 'stă': 32, 'grădină': 33, 'prieteni': 34, 'bucură': 35, 'de': 36, 'timpul': 37, 'petrecut': 38, 'împreună': 39, 'mai': 40, 'mulți': 41, 'căști': 42, 'lucrează': 43, 'sistem': 44, 'uriaș': 45, 'scripete': 46, 'lucrătorii': 47, 'privesc': 48, 'sus': 49, 'jos': 50, 'pe': 51, 'o': 52, 'piesă': 53, 'echipament': 54, 'mașină': 55, 'purtând': 56, 'protecție': 57, 'patru': 58, 'vârful': 59, 'unei': 60, 'structuri': 61, 'înalte': 62, 'trei': 63, 'platformă': 64, 'mare': 65, 'copil': 66, 'rochie': 67, 'roz': 68, 'urcă': 69, 'scară': 70, 'întrun': 71, 'hol': 72, 'fetiță': 73, 'intră': 74, 'cabană': 75, 'lemn': 76, 'scările': 

In [ ]:
from keras.initializers import Constant

def define_model_LSTM_ADD_embs(vocab_size, max_length, in_shape, embed_dim, ff_dim, lstm_dim, dropout, embedding_matrix):
    inputs1 = Input(shape=(in_shape,))
    fe1 = Dropout(dropout)(inputs1)
    fe2 = Dense(ff_dim, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size,
                    embed_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    trainable=False)(inputs2)
    se2 = Dropout(dropout)(se1)
    se3 = LSTM(lstm_dim)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(ff_dim, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    plot_model(model, to_file=f'model_embs.png', show_shapes=True)
    return model

#Caption generation

In [ ]:
def word_for_id(integer, vocab):
    for word, index in vocab.items():
        if index == integer:
            return word
    return None

def generate_desc(model, photo, max_length, vocab):
    # Seed the generation process
    in_text = '('
    # Iterate over the whole length of the sequence
    for i in range(1, max_length):
        # Encode the input sequence
        sequence = in_text.split()
        sequence = [vocab[w] for w in sequence]
        sequence = pad_sequences([sequence], maxlen=max_length)
        # Predict next word

        yhat = model.predict([photo, sequence], verbose=0)  # Reshape photo to match expected shape #photo.reshape(1, -1)
        # Convert probability to integer
        yhat = np.argmax(yhat)
        # Map integer to word
        word = word_for_id(yhat, vocab)
        # Stop if we cannot map the word or if we predict the end of the sequence
        if word is None or word == ')':
            break
        # Append the predicted word to the input for generating the next word
        in_text += ' ' + word
    return in_text.replace("( ", "").replace(" )", "")

def generate_desc_batch(model, photos, max_length, vocab):
    batch_size = len(photos)
    in_texts = ['('] * batch_size
    completed_descriptions = [''] * batch_size
    for i in range(1, max_length):
        sequences = [in_text.split() for in_text in in_texts]
        sequences2 = []
        for sequence in sequences:
          sequence = [vocab[w] for w in sequence]
          sequences2.append(sequence)
        sequences = pad_sequences(sequences2, maxlen=max_length)
        # sequences = np.array(sequences)
        yhat = model.predict([photos, sequences], verbose=0)
        yhat = np.argmax(yhat, axis=1)
        words = [word_for_id(y, vocab) for y in yhat]

        for j in range(batch_size):
            word = words[j]
            if word is None or word == ')':
                continue
            in_texts[j] += ' ' + word
            completed_descriptions[j] = in_texts[j]

    completed_descriptions = [desc.replace("( ", "").replace(' )', "") for desc in completed_descriptions]
    # print(completed_descriptions)
    return completed_descriptions

def batch_process_images(model, vocab, image_paths, batch_size, max_length):
    descriptions = []
    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch_paths = image_paths[i:i + batch_size]
        photo_batch = [preprocess_image(path) for path in batch_paths]
        photo_features_batch = np.array([cnn_model(np.expand_dims(photo, axis=0)) for photo in photo_batch])
        photo_features_batch = np.squeeze(photo_features_batch, axis=1)
        batch_descriptions = generate_desc_batch(model, photo_features_batch, max_length, vocab)
        descriptions.extend(batch_descriptions)
    return descriptions

# Training

# VERIFICA DACA IncV3 e bun la instantiere

In [ ]:
df_results = pd.DataFrame()
BATCH_SIZE = 32
LR = 3e-4
VARIANTS = ["v1b"]
MODEL_NAME = "EfficientNetB4" # ["ResNet50", "ResNet50V2", "InceptionV3", "InceptionResNetV2", "EfficientNetB4", "EfficientNetB7", "VitB16"]

for variant in VARIANTS:
  cnn_model, out_shape = get_cnn_model(MODEL_NAME)
  # cnn_model, out_shape = get_vit_model()
  print(out_shape)
  model = define_model_LSTM_ADD_embs(
    vocab_size,
    SEQ_LENGTH + 2,
    out_shape,
    300,
    FF_DIM,
    LSTM_DIM,
    DROPOUT,
    embedding_matrix)

  train_generator = data_generator(train_descriptions,
                                  cnn_model,
                                  SEQ_LENGTH + 2,
                                  BATCH_SIZE,
                                  IMAGES_PATH,
                                  IMAGE_AUGMENTATION,
                                  vocab_size,
                                  vocab)
  test_generator = data_generator(test_descriptions,
                                  cnn_model,
                                  SEQ_LENGTH + 2,
                                  BATCH_SIZE,
                                  IMAGES_PATH,
                                  IMAGE_AUGMENTATION,
                                  vocab_size,
                                  vocab)


  class PrintDescCallback(Callback):
      def on_epoch_begin(self, epoch, logs=None):
          img_path = f'./flickr30k-images/{list(test_descriptions.keys())[0]}.jpg'
          photo_features = cnn_model(np.expand_dims(preprocess_image(img_path), axis=0))
          desc = generate_desc(self.model, photo_features, SEQ_LENGTH + 2, vocab)
          print("Start epoch {} of training =>".format(epoch))
          print("ORG: {}".format(test_descriptions[list(test_descriptions.keys())[0]][0].replace("startseq ", "").replace(" endseq", "")))
          print("GEN: {}".format(desc))

      def on_train_end(self, epoch, logs=None):
          img_path = f'./flickr30k-images/{list(test_descriptions.keys())[0]}.jpg'
          photo_features = cnn_model(np.expand_dims(preprocess_image(img_path), axis=0))
          desc = generate_desc(self.model, photo_features, SEQ_LENGTH + 2, vocab)
          print("Start epoch {} of training =>".format(epoch))
          print("ORG: {}".format(test_descriptions[list(test_descriptions.keys())[0]][0].replace("startseq ", "").replace(" endseq", "")))
          print("GEN: {}".format(desc))


  steps_per_epoch = len(train_descriptions) // BATCH_SIZE;
  validation_steps = len(test_descriptions) // BATCH_SIZE;
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=LR), metrics=['accuracy'])
  checkpoint = ModelCheckpoint('./drive/MyDrive/LICENTA-MODELS/' + MODEL_NAME + '-' + variant + '-CC-{epoch:02d}.tf', monitor='val_loss', verbose=1, save_best_only=False, mode='min')


  model.fit(
      train_generator,
      validation_data = test_generator,
      epochs = EPOCHS,
      verbose = 1,
      callbacks = [checkpoint, PrintDescCallback()],
      steps_per_epoch = steps_per_epoch,
      validation_steps = validation_steps,
      validation_freq = 1
  )


  image_paths = [os.path.join(IMAGES_PATH, key + '.jpg') for key in test_descriptions.keys()]
  preds = batch_process_images(model, vocab, image_paths, 64, SEQ_LENGTH + 2)

  refs = []
  for descs in list(test_descriptions.values()):
    new_descs = []
    for desc in descs:
      new_descs.append(desc.replace("( ", "").replace(" )", ""))
    refs.append(new_descs)


  results = bleu.compute(predictions=preds, references=refs)
  b1, b2, b3, b4 = results['precisions']
  results = meteor.compute(predictions=preds, references=refs)
  met = results['meteor']
  results = ter.compute(predictions=preds, references=refs)
  te = results['score']
  results = rouge.compute(predictions=preds, references=refs)
  rougeL, r1, r2 = results['rougeL'], results['rouge1'], results['rouge2']
  results = cider(predictions=preds, references=refs, df='corpus')
  cid = results['avg_score']

  new_data = pd.DataFrame({'model': [f"{MODEL_NAME}-{variant}"],
                          'B-1': [b1], 'B-2': [b2], 'B-3': [b3], 'B-4': [b4],
                          'CIDEr': [cid], 'R-1': [r1], 'R-2': [r2], 'Rouge-L': [rougeL], 'Meteor': [met], 'TER': [te]})

  df_results = pd.concat([df_results, new_data], ignore_index=True)
  print(df_results)

1792
Start epoch 0 of training =>
ORG: ( un bărbat cu haine reflectorizante și protecție auditivă conduce un tractor john _ pe o stradă )
GEN: lanțurilor lanțurilor tavi tavi tavi tavi tavi tavi interpretă desfășura interpretă interpretă desfășura interpretă desfășura interpretă interpretă desfășura interpretă desfășura interpretă desfășura interpretă interpretă desfășura interpretă
Epoch 1/10
836/836 [==============================] - ETA: 0s - loss: 5.2513 - accuracy: 0.1645

KeyError: 'shepherd'